In [ ]:
!pip install contractions
!pip install textsearch
!pip install tqdm
import nltk
nltk.download('punkt')

     |████████████████████████████████| 286kB 6.8MB/s 
     |████████████████████████████████| 327kB 21.9MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85407 sha256=ccf66473c5bf7ba0599da66d8f0ffe7b5edcb7068e3617dc50f150def8084578
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_csv("nlp_train.csv")
df.head()

,id,keyword,location,text,target
0,1707,bridge%20collapse,NaN,Ashes 2015: AustraliaÛªs collapse at Trent Br...,0
1,5789,hail,"Carol Stream, Illinois",GREAT MICHIGAN TECHNIQUE CAMP\nB1G THANKS TO @...,1
2,7789,police,Houston,CNN: Tennessee movie theater shooting suspect ...,1
3,8257,rioting,NaN,Still rioting in a couple of hours left until ...,1
4,10656,wounds,Lake Highlands,Crack in the path where I wiped out this morni...,0


In [ ]:
dataset = pd.DataFrame({'Text':df['text'],'Target':df['target']})
dataset.head()

,Text,Target
0,Ashes 2015: AustraliaÛªs collapse at Trent Br...,0
1,GREAT MICHIGAN TECHNIQUE CAMP\nB1G THANKS TO @...,1
2,CNN: Tennessee movie theater shooting suspect ...,1
3,Still rioting in a couple of hours left until ...,1
4,Crack in the path where I wiped out this morni...,0


In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5329 entries, 0 to 5328
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    5329 non-null   object
 1   Target  5329 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 83.4+ KB


In [ ]:
# build train and test datasets
text = dataset['Text'].values
labels = dataset['Target'].values

train_text = text[:3500]
train_labels = labels[:3500]

test_text = text[3500:]
test_labels = labels[3500:]

In [ ]:
import contractions
from bs4 import BeautifulSoup
import numpy as np
import re
import tqdm
import unicodedata


def strip_html_tags(text):
  soup = BeautifulSoup(text, "html.parser")
  [s.extract() for s in soup(['iframe', 'script'])]
  stripped_text = soup.get_text()
  stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
  return stripped_text

def remove_accented_chars(text):
  text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
  return text

def remove_http(url):
  p= re.compile('(http(s)?|www)')
  url = p.sub('',url)
  return url

def remove_stopwords(text, is_lower_case=False, stopwords=None):
    if not stopwords:
        stopwords = nltk.corpus.stopwords.words('english')
    tokens = nltk.word_tokenize(text)
    tokens = [token.strip() for token in tokens]
    
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopwords]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopwords]
    
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

def pre_process_corpus(docs):
  norm_docs = []
  for doc in tqdm.tqdm(docs):
    doc = strip_html_tags(doc)
    doc = doc.translate(doc.maketrans("\n\t\r", "   "))
    doc = doc.lower()
    doc = remove_stopwords(doc)
    doc = remove_accented_chars(doc)
    doc = contractions.fix(doc)
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = re.sub(' +', ' ', doc)
    doc = doc.strip()  
    doc = remove_http(doc)
    norm_docs.append(doc)
  
  return norm_docs

In [ ]:
%%time

norm_train_reviews = pre_process_corpus(train_text)
norm_test_reviews = pre_process_corpus(test_text)


100%|██████████| 1829/1829 [00:01<00:00, 1383.59it/s]

CPU times: user 3.64 s, sys: 218 ms, total: 3.86 s
Wall time: 3.96 s


In [ ]:
norm_train_reviews[:10]

['ashes 2015 australiauas collapse trent bridge among worst history england bundled australia 60  tcot5trhjuau0',
 'great michigan technique camp b1g thanks bmurph1019 hailyoutsey termn8r13 goblue wrestleon  tcooaskgki6qj',
 'cnn tennessee movie theater shooting suspect killed police  tcodi8elzswnr',
 'still rioting couple hours left class',
 'crack path wiped morning beach run surface wounds left elbow right knee  tcoyaqrsximph',
 'experts france begin examining airplane debris found reunion island french air accident experts  tcotagzbcxfj0 mlb',
 'i came kill indians fun video smirking remorseless pakistani killer shows boasting  tcofpjlwoxklg',
 'johnsontionne except idk s really burning',
 'destroy house',
 'police officer wounded suspect dead exchanging shots  tcoxxfk4khbiw']

In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(train_labels),
                                                 train_labels)
class_weight_dict = dict(enumerate(class_weights))
class_weight_dict

{0: 0.8745627186406797, 1: 1.1674449633088726}

In [ ]:

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# build BOW features on train reviews
cv = CountVectorizer(binary=False, min_df=5, max_df=1.0, ngram_range=(1,2))
cv_train_features = cv.fit_transform(norm_train_reviews)


# build TFIDF features on train reviews
tv = TfidfVectorizer(use_idf=True, min_df=5, max_df=1.0, ngram_range=(1,2),
                     sublinear_tf=True)
tv_train_features = tv.fit_transform(norm_train_reviews)

In [ ]:
# transform test reviews into features
cv_test_features = cv.transform(norm_test_reviews)
tv_test_features = tv.transform(norm_test_reviews)

In [ ]:
print('BOW model:> Train features shape:', cv_train_features.shape, ' Test features shape:', cv_test_features.shape)
print('TFIDF model:> Train features shape:', tv_train_features.shape, ' Test features shape:', tv_test_features.shape)

BOW model:> Train features shape: (3500, 1855)  Test features shape: (1829, 1855)
TFIDF model:> Train features shape: (3500, 1855)  Test features shape: (1829, 1855)


In [ ]:
#class_weight_dict = {0: 1.,1: 5.}

In [ ]:
%%time
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV


dual=[False]
#max_iter=[100,200,300,400,500]
C = [0.1, 1, 10,100]
penalty = ['l1','l2']
param_grid = dict(dual=dual,C=C,penalty=penalty)

lr = LogisticRegression(solver='liblinear',class_weight=class_weight_dict)
grid = GridSearchCV(estimator=lr, param_grid=param_grid, cv = 5)

grid_result = grid.fit(cv_train_features, train_labels)
#grid_tfidf_result = grid.fit(tv_train_features, train_labels)
# Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
#print("Best: %f using %s" % (grid_tfidf_result.best_score_, grid_tfidf_result.best_params_))


Best: 0.783429 using {'C': 0.1, 'dual': False, 'penalty': 'l2'}
CPU times: user 1.94 s, sys: 1.71 ms, total: 1.94 s
Wall time: 1.95 s


In [ ]:
lr_bow_predictions = grid.predict(cv_test_features)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

labels = ['negative', 'positive']
print(classification_report(test_labels, lr_bow_predictions))
pd.DataFrame(confusion_matrix(test_labels, lr_bow_predictions), index=labels, columns=labels)

              precision    recall  f1-score   support

           0       0.77      0.84      0.80      1023
           1       0.77      0.68      0.72       806

    accuracy                           0.77      1829
   macro avg       0.77      0.76      0.76      1829
weighted avg       0.77      0.77      0.76      1829



,negative,positive
negative,856,167
positive,260,546


In [ ]:
%%time
grid_tfidf_result = grid.fit(tv_train_features, train_labels)
print("Best: %f using %s" % (grid_tfidf_result.best_score_, grid_tfidf_result.best_params_))


Best: 0.778000 using {'C': 0.1, 'dual': False, 'penalty': 'l2'}
CPU times: user 887 ms, sys: 6.36 ms, total: 893 ms
Wall time: 900 ms


In [ ]:
lr_tfidf_predictions = grid.predict(tv_test_features)

In [ ]:

labels = ['negative', 'positive']
print(classification_report(test_labels, lr_tfidf_predictions))
pd.DataFrame(confusion_matrix(test_labels, lr_tfidf_predictions), index=labels, columns=labels)

              precision    recall  f1-score   support

           0       0.78      0.80      0.79      1023
           1       0.74      0.72      0.73       806

    accuracy                           0.76      1829
   macro avg       0.76      0.76      0.76      1829
weighted avg       0.76      0.76      0.76      1829



,negative,positive
negative,818,205
positive,225,581


In [ ]:
%%time
from sklearn.model_selection import RandomizedSearchCV
random = RandomizedSearchCV(estimator=lr, param_distributions=param_grid, cv = 5)


random_result = random.fit(tv_train_features, train_labels)
# Summarize results
print("Best: %f using %s" % (random_result.best_score_, random_result.best_params_))


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


Best: 0.778000 using {'penalty': 'l2', 'dual': False, 'C': 0.1}
CPU times: user 943 ms, sys: 1.82 ms, total: 945 ms
Wall time: 949 ms


In [ ]:
%%time
from sklearn.svm import SVC

kernels = ['linear', 'rbf', 'poly']
C= [0.1, 1, 10,100]
degrees = [1, 2, 3]
param_grid = dict(kernel=kernels,C=C,degree=degrees)

model = SVC(class_weight=class_weight_dict)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv = 5)

grid_result = grid.fit(cv_train_features, train_labels)

# Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


Best: 0.789143 using {'C': 0.1, 'degree': 1, 'kernel': 'linear'}
CPU times: user 2min 40s, sys: 302 ms, total: 2min 41s
Wall time: 2min 40s


In [ ]:
svc_predictions = grid.predict(cv_test_features)

In [ ]:
labels = ['negative', 'positive']
print(classification_report(test_labels, svc_predictions))
pd.DataFrame(confusion_matrix(test_labels, svc_predictions), index=labels, columns=labels)

              precision    recall  f1-score   support

           0       0.75      0.86      0.80      1023
           1       0.79      0.63      0.70       806

    accuracy                           0.76      1829
   macro avg       0.77      0.75      0.75      1829
weighted avg       0.76      0.76      0.76      1829



,negative,positive
negative,884,139
positive,298,508


In [ ]:
%%time
svc_tfidf_result = grid.fit(tv_train_features, train_labels)
print("Best: %f using %s" % (svc_tfidf_result.best_score_, svc_tfidf_result.best_params_))

Best: 0.786857 using {'C': 0.1, 'degree': 1, 'kernel': 'poly'}
CPU times: user 2min 30s, sys: 230 ms, total: 2min 31s
Wall time: 2min 30s


In [ ]:
svc_tfidf_predictions = grid.predict(tv_test_features)

In [ ]:
labels = ['negative', 'positive']
print(classification_report(test_labels, svc_tfidf_predictions))
pd.DataFrame(confusion_matrix(test_labels, svc_tfidf_predictions), index=labels, columns=labels)

              precision    recall  f1-score   support

           0       0.76      0.85      0.80      1023
           1       0.78      0.66      0.71       806

    accuracy                           0.77      1829
   macro avg       0.77      0.76      0.76      1829
weighted avg       0.77      0.77      0.76      1829



,negative,positive
negative,871,152
positive,275,531


In [ ]:
%%time
from xgboost.sklearn import XGBClassifier


max_depth=[10,15,20]
n_estimator = [100,200,300,400,500]
#min_child_weight=[2,4,6,8,10]
param_grid = dict(max_depth=max_depth,n_estimator=n_estimator)

grid = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27,class_weight=class_weight_dict),param_grid = param_grid, cv=5)

xgb_result = grid.fit(cv_train_features, train_labels)

# Summarize results
print("Best: %f using %s" % (xgb_result.best_score_, xgb_result.best_params_))

Best: 0.776000 using {'max_depth': 15, 'n_estimator': 100}
CPU times: user 2min 47s, sys: 6.99 s, total: 2min 54s
Wall time: 1min 32s


In [ ]:
xgb_predictions = grid.predict(cv_test_features)

In [ ]:
labels = ['negative', 'positive']
print(classification_report(test_labels, xgb_predictions))
pd.DataFrame(confusion_matrix(test_labels, xgb_predictions), index=labels, columns=labels)

              precision    recall  f1-score   support

           0       0.74      0.88      0.80      1023
           1       0.80      0.60      0.69       806

    accuracy                           0.76      1829
   macro avg       0.77      0.74      0.74      1829
weighted avg       0.77      0.76      0.75      1829



,negative,positive
negative,904,119
positive,324,482


In [ ]:
%%time
xgb_tfidf_result = grid.fit(tv_train_features, train_labels)
print("Best: %f using %s" % (xgb_tfidf_result.best_score_, xgb_tfidf_result.best_params_))

Best: 0.766857 using {'max_depth': 20, 'n_estimator': 100}
CPU times: user 3min 44s, sys: 7.94 s, total: 3min 52s
Wall time: 2min 1s


In [ ]:
xgb_tfidf_predictions = grid.predict(tv_test_features)

In [ ]:
labels = ['negative', 'positive']
print(classification_report(test_labels, xgb_tfidf_predictions))
pd.DataFrame(confusion_matrix(test_labels, xgb_tfidf_predictions), index=labels, columns=labels)

              precision    recall  f1-score   support

           0       0.74      0.88      0.80      1023
           1       0.80      0.61      0.69       806

    accuracy                           0.76      1829
   macro avg       0.77      0.74      0.75      1829
weighted avg       0.77      0.76      0.75      1829



,negative,positive
negative,898,125
positive,313,493
